In [1]:
from nltk import pos_tag
from sklearn.pipeline import Pipeline

from data_access.twitter_reader import TwitterReader
from data_storage.db_manager import DBManager
from data_wrangling.wrangler import Wrangler
from data_transformers.normalizers import TextNormalizer
from data_transformers.vectorizer import OneHotVectorizer
from models.models import KMeansClusters
from sklearn.metrics import accuracy_score

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

In [ ]:
# db_manager = DBManager()
# db_manager.reset_raw_tweets()
# db_manager.reset_processed_tweets()

In [2]:
db_manager = DBManager()

# q_dict = {'#': ['hiphop']}
# tr = TwitterReader('./local_configs/api_keys.json')
# tr.set_query(q_dict)
# result = tr.read(pages=10)

# db_manager.save_tweets(result)

In [17]:
processed_tweets = list(db_manager.get_processed_tweets())

In [22]:
tn = TextNormalizer()
tn.transform(processed_tweets)

[('#NoRegretz', 'JJ'), ('#MobbMarley', 'NNP'), ('#Flexin', 'NNP'), ('&', 'CC'), ('#Finessin', 'NNP'), ('#Remix', 'NNP'), ('to', 'TO'), ('#Flexin', 'VB'), ('#LilWayne', 'NNP'), ('#MobbsterRemix', 'NNP'), ('#Click', 'NNP'), ('the', 'DT'), ('#Link', 'NN'), ('in', 'IN'), ('#MyBio', '$'), ('2', 'CD'), ('#Hear', 'JJ'), ('my', 'PRP$'), ('#Mixtape', 'NN'), ('#Hiphop', 'NN'), ('#Music', 'JJ'), ('#MTV', 'NNP'), ('#Vh1', 'NNP'), ('#BET', 'NNP'), ('#MoBBMarley', 'NNP'), ('#JayPayso', 'NNP'), ('#RastaSinatra', 'NNP'), ('#Batman', 'NN')]
[('Paris', 'NNP'), ('-', ':'), ('Walk', 'VBP'), ('Like', 'IN'), ('A', 'NNP'), ('Panther', 'NNP'), ('(', '('), ('2020', 'CD'), (')', ')'), ('From', 'IN'), ('the', 'DT'), ('Album', 'NNP'), ('Safe', 'NNP'), ('Space', 'NNP'), ('Invader', 'NNP'), ('.', '.'), ('#Rap', 'NN'), ('#HipHop', 'NNP'), ('#YouTube', 'NNP'), ('#WestCoast', 'NNP'), ('#NowPlaying', 'VBG'), ('#Paris', 'JJ'), ('#GuerrillaFunkRecordings', 'NNS'), ('#FilmworksLLC', 'VBP'), ('#MusicBlog', 'CD'), ('#MusikB

999

In [ ]:
db_manager.save_processed_tweets(wrangler.tokenize())

In [ ]:
next(db_manager.get_tweet_texts(text_only=True))

In [ ]:
from yellowbrick.text import TSNEVisualizer
from sklearn.feature_extraction.text import TfidfVectorizer

data_text = list(db_manager.get_tweet_texts(text_only=True))


tfidf = TfidfVectorizer()
docs = tfidf.fit_transform(data_text)

tsne = TSNEVisualizer()
tsne.fit(docs)
tsne.poof()

In [ ]:
# Apply clustering instead of class names.
from sklearn.cluster import KMeans
clusters = KMeans(n_clusters=2)
clusters.fit(docs)

clustered_docs = zip(data_text, clusters.labels_)

tsne = TSNEVisualizer()
tsne.fit(docs, clusters.labels_)
tsne.poof()

In [ ]:
next(filter(lambda x: x[1] == 0, clustered_docs))

In [ ]:
db_manager = DBManager()
vectorizer = OneHotVectorizer()
g = db_manager.get_processed_tweets()

model = Pipeline([
    ('norm', TextNormalizer()),
    ('vect', OneHotVectorizer()),
    ('clusters', KMeansClusters(k=2))
])

clusters = model.fit_transform(g)

In [ ]:
accuracy_score([0]*998, clusters[:998])

In [ ]:
accuracy_score([1]*1000, clusters[998:])

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
g = list(db_manager.get_processed_tweets())[998:]

n_topics = 5
model = Pipeline([
    ('norm', TextNormalizer()),
    ('vect', CountVectorizer(
    preprocessor=None, lowercase=False)),
    ('model', LatentDirichletAllocation(n_components=n_topics)),
])

model.fit_transform(g)

In [ ]:
vectorizer = model.named_steps['vect']
model = model.steps[-1][1]
names = vectorizer.get_feature_names()
topics = dict()

for idx, topic in enumerate(model.components_):
    features = topic.argsort()[:-(10 - 1): -1]
    tokens = [names[i] for i in features]
    topics[idx] = tokens

In [ ]:
topics

In [ ]:
ny_tweets = list(db_manager.get_processed_tweets())[998:]

In [ ]:
def find_token(tweet):
    for t in tweet:
        if 'help' in t[0]:
            return True
    return False

In [ ]:
l = list(filter(find_token, ny_tweets))

In [ ]:
l

In [ ]:
len(clusters)